In [67]:
# Import data from mongodb
from pymongo import MongoClient
MONGO_HOST= 'mongodb://localhost/twitterdb' 
client = MongoClient(MONGO_HOST)
db = client.twitterdb
cursor = db.twitter_search


# Save to a dataframe
import pandas as pd
twitterdf = pd.DataFrame(list(cursor.find()))
twitterdf = twitterdf.loc[twitterdf.lang =='en',:]

### 处理客户端

In [65]:
def sourceInfo(rawdata):
    
    # filter language
    rawdata = rawdata.loc[rawdata.lang =='en',:]
    

    data = rawdata.source.str.extractall(r'(?P<link>\<.+\>)(?P<text>.+)(<.+>)')
    source =  data.groupby('text').count().link 

    # # offical source
    offical = ['Twitter Web Client','Twitter for Android','Twitter for iPad','Twitter for iPhone','Other']

    source_mod = source.loc[source.index.isin(offical)]

    # # classify tweets from other sources as other

    other = pd.Series([source.loc[~source.index.isin(offical)].sum()], index=['Other'])
    source_mod = source_mod.append(other)
    return source_mod 

sourceInfo(twitterdf)


Twitter Web Client       67
Twitter for Android     123
Twitter for iPad         13
Twitter for iPhone     1690
Other                   102
dtype: int64

### LDA

In [102]:
from  Lda_text_format import filter_tweet

# to tower case + remove stop words
clean = twitterdf.text.apply(lambda x: filter_tweet(x))

# remove duplicated tweets
cleaned = clean[~clean.apply(lambda x: ' '.join(word for word in x)).duplicated()]

cleaned = cleaned.apply(lambda x: ' '.join(word for word in x))
print(cleaned.iloc[1])

corpus=[]
a=[]
for i in range(len(cleaned)):
    a= cleaned.iloc[i]

    corpus.append(a)

texts = [[word for word in str(document).lower().split()] for document in corpus]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# lda model
from gensim import corpora, models, similarities
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

total_topics = 5
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

#Show first n important word in the topics:
lda.show_topics(total_topics,5)

In [147]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
import pandas as pd
df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)
df_lda




(82, 5)
(5, 82)


,account,airpod,airpods,apple,battery,beat,bed,big,bitch,bought,...,walking,want,watch,wear,wearing,week,win,wire,wonpil,😂😂😂
0,0.000000,0.00000,0.080891,0.005337,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.003654,0.00000,0.003655,0.003736,0.000000,0.005255,0.000000,0.000000
1,0.000000,0.00579,0.128224,0.006990,0.000000,0.000000,0.000000,0.004666,0.000000,0.005881,...,0.003565,0.008021,0.000000,0.00000,0.000000,0.000000,0.005800,0.000000,0.000000,0.000000
2,0.000000,0.00000,0.082750,0.004281,0.004231,0.000000,0.000000,0.000000,0.004249,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.004228,0.000000
3,0.000000,0.00000,0.147107,0.027578,0.000000,0.003764,0.000000,0.000000,0.000000,0.005933,...,0.000000,0.006207,0.007202,0.00449,0.000000,0.000000,0.004493,0.000000,0.000000,0.000000
4,0.004707,0.00000,0.066461,0.000000,0.000000,0.000000,0.004705,0.000000,0.000000,0.002610,...,0.000000,0.025284,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002826


In [148]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
pyLDAvis.save_html(panel, 'lda.html')
panel



PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      59.819107        1       1  206.065399  -35.955723
3      10.070930        1       2   21.799252 -281.204407
2      10.039500        1       3 -111.189186  229.390869
0      10.038184        1       4 -206.868744  -61.136806
4      10.032269        1       5  203.454926  270.803497, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
0     Default  57.000000    airpods  57.000000  30.0000  30.0000
60    Default   4.000000       want   4.000000  29.0000  29.0000
14    Default   4.000000      apple   4.000000  28.0000  28.0000
250   Default   1.000000       talk   1.000000  27.0000  27.0000
116   Default   1.000000     iphone   1.000000  26.0000  26.0000
188   Default   2.000000        buy   2.000000  25.0000  25.0000
470   Default   1.000000        lol   1.000000  24.0000  24.0000
146   Default   1.000000       need   1.000000  23.0000  23.0000
283   Default   0.000000       lmao   0.000000  22.0000  22.0000
29    Default   0.000000    fucking   0.000000  21.0000  21.0000
16    Default   1.000000      nigga   1.000000  20.0000  20.0000
68    Default   1.000000        ear   1.000000  19.0000  19.0000
20    Default   1.000000    getting   1.000000  18.0000  18.0000
100   Default   1.000000      think   1.000000  17.0000  17.0000
177   Default   1.000000       love   1.000000  16.0000  16.0000
220   Default   1.000000      watch   1.000000  15.0000  15.0000
97    Default   4.000000       lost   4.000000  14.0000  14.0000
85    Default   1.000000       feel   1.000000  13.0000  13.0000
21    Default   1.000000        gon   1.000000  12.0000  12.0000
36    Default   1.000000  headphone   1.000000  11.0000  11.0000
258   Default   0.000000   thinking   0.000000  10.0000  10.0000
42    Default   1.000000       wait   1.000000   9.0000   9.0000
183   Default   0.000000        put   0.000000   8.0000   8.0000
461   Default   0.000000       shit   0.000000   7.0000   7.0000
108   Default   1.000000     listen   1.000000   6.0000   6.0000
115   Default   2.000000     bought   2.000000   5.0000   5.0000
635   Default   0.000000     doctor   0.000000   4.0000   4.0000
480   Default   0.000000      going   0.000000   3.0000   3.0000
22    Default   1.000000       hear   1.000000   2.0000   2.0000
126   Default   2.000000       look   2.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
90     Topic5   0.130024      train   0.241539   1.6800  -5.9652
829    Topic5   0.130015       bro…   0.241528   1.6800  -5.9653
629    Topic5   0.130046       bank   0.241585   1.6800  -5.9651
632    Topic5   0.130042  obviously   0.241578   1.6800  -5.9651
60     Topic5   1.280980       want   4.062650   1.1452  -3.6776
108    Topic5   0.347199     listen   1.090254   1.1551  -4.9831
0      Topic5   3.367129    airpods  57.879475  -0.5449  -2.7111
258    Topic5   0.239038   thinking   0.682110   1.2508  -5.3563
238    Topic5   0.238355        bed   0.687026   1.2408  -5.3592
873    Topic5   0.143179        😂😂😂   0.288552   1.5986  -5.8689
58     Topic5   0.238255       damn   0.761172   1.1378  -5.3596
146    Topic5   0.346868       need   1.721153   0.6976  -4.9840
66     Topic5   0.203735     trying   0.603750   1.2130  -5.5161
38     Topic5   0.237854     person   0.836616   1.0417  -5.3613
480    Topic5   0.237414      going   0.855814   1.0171  -5.3631
42     Topic5   0.294927       wait   1.545954   0.6427  -5.1462
21     Topic5   0.238816        gon   1.090128   0.7810  -5.3573
85     Topic5   0.238359       feel   1.582581   0.4063  -5.3592
392    Topic5   0.238044  christmas   2.106629   0.1190  -5.3605
100    Topic5   0.196865      think   1.604353   0.2014  -5.5504
116    Topic5   0.187329     iphone   1.459006   0.2467  -5.6001
587    Topic5   0.130280        nct   0.2